In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [5]:
import torch.nn.functional as F  # useful stateless functions

def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass; 
    PyTorch will take care of the backward pass for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).
    
    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2 = params
    
    # Forward pass: compute predicted y using operations on Tensors. Since w1 and
    # w2 have requires_grad=True, operations involving these Tensors will cause
    # PyTorch to build a computational graph, allowing automatic computation of
    # gradients. Since we are no longer implementing the backward pass by hand we
    # don't need to keep references to intermediate values.
    # you can also use `.clamp(min=0)`, equivalent to F.relu()
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x
    

def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # you should see [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


In [6]:
def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?
    
    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    ################################################################################
    l1_output = F.relu ( F.conv2d(x, conv_w1, conv_b1, padding = 2) )
    
    l2_output = F.relu ( F.conv2d(l1_output, conv_w2, conv_b2, padding = 1) )
    scores = flatten(l2_output).mm(fc_w) + fc_b
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

In [7]:
def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # you must calculate the shape of the tensor after two conv layers, before the fully-connected layer
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # you should see [64, 10]
three_layer_convnet_test()

torch.Size([64, 10])


In [8]:
filters = torch.randn(3, 16, 2)
inputs = torch.randn(27, 16, 50)
F.conv1d(inputs, filters).shape

torch.Size([27, 3, 49])

In [9]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[-0.3182, -0.6446, -0.8752, -0.8429, -0.9213],
        [-1.3608, -0.5483,  0.4071,  0.4664,  0.1871],
        [ 0.5136,  0.3998, -0.7567,  0.7409,  0.2748]], device='cuda:0',
       requires_grad=True)

In [13]:
def check_accuracy_part2(loader, model_fn, params):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
#             print(y)
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

In [11]:
def train_part2(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD
    
    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
#                 print(type(w.grad))
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()
            

In [12]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)


Iteration 0, loss = 3.1382
Checking accuracy on the val set
tensor([0, 1, 1, 7, 4, 5, 9, 8, 1, 8, 3, 7, 1, 7, 1, 1, 8, 5, 5, 9, 9, 8, 9, 8,
        1, 9, 7, 5, 7, 3, 6, 3, 8, 3, 5, 1, 8, 1, 8, 4, 0, 0, 2, 3, 9, 8, 1, 3,
        6, 8, 8, 1, 3, 0, 6, 0, 5, 1, 2, 0, 8, 9, 4, 8])
tensor([1, 2, 9, 3, 8, 8, 3, 4, 2, 9, 1, 2, 7, 2, 3, 7, 6, 5, 5, 3, 5, 3, 7, 6,
        1, 4, 9, 5, 7, 3, 2, 1, 6, 6, 7, 8, 6, 7, 4, 9, 1, 2, 6, 5, 5, 2, 3, 1,
        3, 8, 7, 1, 8, 6, 8, 9, 2, 4, 5, 1, 3, 9, 1, 8])
tensor([5, 3, 2, 8, 0, 8, 9, 3, 6, 1, 5, 6, 3, 8, 1, 6, 3, 0, 9, 2, 9, 8, 3, 7,
        5, 7, 2, 1, 0, 9, 6, 3, 4, 1, 2, 5, 7, 3, 7, 8, 9, 6, 0, 9, 3, 8, 1, 5,
        7, 7, 5, 6, 3, 3, 4, 5, 9, 3, 7, 0, 3, 3, 6, 7])
tensor([2, 4, 2, 7, 8, 0, 9, 2, 9, 2, 4, 2, 7, 3, 2, 8, 5, 0, 3, 2, 7, 1, 1, 5,
        3, 2, 0, 8, 8, 7, 0, 7, 3, 8, 9, 7, 2, 9, 5, 9, 5, 4, 8, 5, 5, 1, 6, 6,
        0, 9, 1, 2, 7, 0, 0, 7, 1, 5, 4, 0, 3, 3, 7, 5])
tensor([6, 6, 9, 6, 9, 5, 4, 5, 7, 3, 8, 8, 3, 7, 4, 9, 2, 0, 7, 2, 3, 7

Iteration 300, loss = 1.8772
Checking accuracy on the val set
tensor([3, 7, 4, 6, 8, 5, 4, 7, 2, 8, 6, 6, 1, 3, 2, 4, 5, 0, 9, 6, 5, 9, 1, 8,
        3, 9, 5, 0, 9, 3, 0, 7, 3, 9, 8, 8, 3, 4, 7, 8, 0, 3, 0, 7, 4, 9, 8, 0,
        9, 5, 3, 9, 6, 6, 9, 9, 0, 8, 6, 6, 2, 3, 8, 4])
tensor([1, 4, 7, 7, 7, 1, 1, 7, 7, 3, 1, 8, 3, 5, 6, 5, 5, 0, 6, 7, 5, 4, 5, 9,
        1, 8, 5, 6, 7, 8, 1, 5, 7, 9, 3, 4, 8, 9, 7, 5, 4, 8, 3, 1, 3, 2, 5, 7,
        4, 1, 6, 1, 4, 7, 6, 5, 5, 7, 7, 6, 8, 0, 2, 6])
tensor([5, 5, 3, 5, 1, 4, 9, 9, 5, 1, 1, 1, 0, 0, 7, 1, 6, 5, 0, 8, 9, 4, 0, 4,
        7, 5, 1, 5, 8, 6, 3, 5, 7, 6, 9, 3, 3, 5, 3, 3, 0, 0, 6, 4, 4, 0, 4, 0,
        3, 3, 7, 9, 1, 0, 8, 4, 7, 4, 2, 7, 2, 1, 6, 1])
tensor([8, 7, 1, 9, 9, 3, 6, 5, 5, 2, 9, 4, 1, 9, 5, 2, 4, 2, 6, 1, 1, 5, 6, 6,
        3, 4, 0, 4, 2, 8, 3, 2, 8, 3, 1, 9, 7, 3, 1, 8, 3, 3, 0, 4, 5, 1, 1, 4,
        9, 0, 1, 7, 5, 6, 6, 0, 3, 8, 9, 9, 7, 8, 3, 6])
tensor([1, 2, 9, 0, 8, 3, 6, 0, 7, 1, 8, 0, 0, 3, 5, 4, 5, 4, 0, 0, 1,

Iteration 600, loss = 1.5638
Checking accuracy on the val set
tensor([4, 4, 6, 6, 7, 9, 7, 8, 7, 6, 4, 2, 3, 8, 8, 8, 3, 1, 7, 0, 3, 6, 5, 1,
        2, 1, 1, 5, 6, 9, 2, 3, 7, 7, 4, 3, 2, 8, 9, 1, 0, 6, 9, 6, 7, 9, 9, 9,
        5, 1, 1, 1, 7, 5, 4, 7, 5, 6, 3, 1, 5, 6, 5, 4])
tensor([9, 3, 5, 7, 1, 6, 8, 8, 1, 3, 8, 5, 6, 6, 9, 3, 5, 6, 1, 7, 9, 7, 7, 1,
        9, 2, 0, 0, 5, 9, 1, 7, 7, 7, 3, 7, 8, 2, 8, 8, 3, 7, 2, 6, 2, 3, 0, 9,
        8, 5, 6, 7, 7, 3, 4, 4, 6, 4, 3, 7, 9, 4, 0, 7])
tensor([1, 6, 6, 6, 0, 8, 0, 7, 7, 1, 9, 5, 3, 0, 2, 2, 8, 8, 1, 2, 9, 4, 1, 0,
        7, 0, 0, 1, 0, 1, 5, 8, 7, 6, 3, 3, 5, 3, 0, 2, 8, 6, 8, 8, 6, 8, 8, 9,
        2, 2, 7, 8, 7, 9, 4, 0, 9, 7, 8, 9, 9, 2, 0, 7])
tensor([8, 3, 9, 1, 3, 1, 9, 0, 5, 3, 0, 4, 7, 6, 1, 4, 3, 8, 0, 9, 5, 4, 6, 6,
        4, 9, 9, 9, 1, 3, 9, 0, 2, 4, 7, 9, 0, 6, 7, 0, 0, 8, 2, 8, 3, 5, 3, 2,
        5, 6, 4, 8, 9, 9, 1, 5, 3, 5, 8, 4, 8, 6, 8, 7])
tensor([1, 1, 8, 4, 0, 9, 4, 9, 0, 3, 3, 9, 6, 4, 6, 5, 6, 3, 9, 6, 0,

In [19]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
################################################################################
conv_w1 = random_weight((channel_1,3,5,5))
conv_b1 = torch.randn(channel_1, device = device , dtype = dtype)
conv_w2 = random_weight((channel_2,channel_1,3,3))
conv_b2 = torch.randn(channel_2, device = device , dtype = dtype)
fc_w = torch.zeros(( channel_2* 32 * 32, 10) , device = device , dtype = dtype)
fc_b = torch.zeros((10,), device = device , dtype = dtype)

conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight((channel_1,))
conv_w2 = random_weight((channel_2, channel_1, 3, 3))
conv_b2 = zero_weight((channel_2,))
fc_w = random_weight((32 * 32 * 16, 10))
fc_b = zero_weight((10,))
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

Iteration 0, loss = 3.8019
Checking accuracy on the val set
Got 117 / 1000 correct (11.70%)

Iteration 100, loss = 1.9322
Checking accuracy on the val set
Got 367 / 1000 correct (36.70%)

Iteration 200, loss = 1.8996
Checking accuracy on the val set
Got 413 / 1000 correct (41.30%)

Iteration 300, loss = 1.3932
Checking accuracy on the val set
Got 439 / 1000 correct (43.90%)

Iteration 400, loss = 1.6411
Checking accuracy on the val set
Got 444 / 1000 correct (44.40%)

Iteration 500, loss = 1.4835
Checking accuracy on the val set
Got 452 / 1000 correct (45.20%)

Iteration 600, loss = 1.8339
Checking accuracy on the val set
Got 491 / 1000 correct (49.10%)

Iteration 700, loss = 1.5094
Checking accuracy on the val set
Got 505 / 1000 correct (50.50%)



In [20]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init package contains convenient initialization methods
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward always defines connectivity
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_TwoLayerFC()

torch.Size([64, 10])


In [32]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        self.conv1 = nn.Conv2d(in_channel, channel_1, 5, padding =2 )
        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, padding = 1)
        self.fc1 = nn.Linear(32*32*channel_2, num_classes)
        
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.fc1.weight)
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        l1 = F.relu(self.conv1(x))
        l2 = F.relu(self.conv2(l1))
        l3 = flatten(l2)
        scores = self.fc1(l3)
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores

In [33]:
def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerConvNet()

torch.Size([64, 10])


In [34]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [43]:
def train_part34(model, optimizer, epochs=4):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [42]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 3.3129
Checking accuracy on validation set
Got 199 / 1000 correct (19.90)

Iteration 100, loss = 38.3055
Checking accuracy on validation set
Got 255 / 1000 correct (25.50)

Iteration 200, loss = 44.0317
Checking accuracy on validation set
Got 260 / 1000 correct (26.00)

Iteration 300, loss = 63.5635
Checking accuracy on validation set
Got 256 / 1000 correct (25.60)

Iteration 400, loss = 122.8695
Checking accuracy on validation set
Got 255 / 1000 correct (25.50)

Iteration 500, loss = 113.5665
Checking accuracy on validation set
Got 248 / 1000 correct (24.80)

Iteration 600, loss = 76.2662
Checking accuracy on validation set
Got 275 / 1000 correct (27.50)

Iteration 700, loss = 146.4477
Checking accuracy on validation set
Got 295 / 1000 correct (29.50)

Iteration 0, loss = 103.2887
Checking accuracy on validation set
Got 302 / 1000 correct (30.20)

Iteration 100, loss = 154.6135
Checking accuracy on validation set
Got 288 / 1000 correct (28.80)

Iteration 200, loss 

In [44]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
################################################################################
model = ThreeLayerConvNet(3, channel_1, channel_2, 10)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 3.1270
Checking accuracy on validation set
Got 120 / 1000 correct (12.00)

Iteration 100, loss = 2.2448
Checking accuracy on validation set
Got 184 / 1000 correct (18.40)

Iteration 200, loss = 2.1137
Checking accuracy on validation set
Got 237 / 1000 correct (23.70)

Iteration 300, loss = 1.9382
Checking accuracy on validation set
Got 325 / 1000 correct (32.50)

Iteration 400, loss = 1.8361
Checking accuracy on validation set
Got 364 / 1000 correct (36.40)

Iteration 500, loss = 1.6496
Checking accuracy on validation set
Got 411 / 1000 correct (41.10)

Iteration 600, loss = 1.6604
Checking accuracy on validation set
Got 402 / 1000 correct (40.20)

Iteration 700, loss = 1.3997
Checking accuracy on validation set
Got 447 / 1000 correct (44.70)

Iteration 0, loss = 1.2186
Checking accuracy on validation set
Got 473 / 1000 correct (47.30)

Iteration 100, loss = 1.3612
Checking accuracy on validation set
Got 471 / 1000 correct (47.10)

Iteration 200, loss = 1.3154
Check